# Prompt Shield Target Documentation + Tutorial

This is a brief tutorial and documentation on using the Prompt Shield Target

## 0 How Prompt Shield Works

### TL;DR

Below is a very quick summary of how Prompt Shield works. You can visit the following links to learn more:\
(Docs): https://learn.microsoft.com/en-us/azure/ai-services/content-safety/concepts/jailbreak-detection\
(Quickstart Guide): https://learn.microsoft.com/en-us/azure/ai-services/content-safety/quickstart-jailbreak

You will need to deploy a content safety endpoint on Azure to use this if you haven't already.

For PyRIT, you can use Prompt Shield as a target, or you can use it as a true/false scorer to see if it detected a jailbreak in your prompt.

### How It Works in More Detail

Prompt Shield is a Content Safety resource that detects attacks (jailbreaks) in the prompts it is given.

The body of the HTTP request you send to it looks like this:
```json
{
    {"userPrompt"}: "this-is-a-user-prompt-as-a-string",
    {"documents"}: [
        "here-is-one-document-as-a-string",
        "here-is-another"
    ]
}
```

And it returns the following in its response:
```json
{
    {"userPromptAnalysis"}: {"attackDetected": "true or false"},
    {"documentsAnalysis"}: [
        {"attackDetected": "true or false for the first document"},
        {"attackDetected": "true or false for the second document"}
    ]
}
```

This document has an example below.

Some caveats and tips:
* You can send any string you like to either of those two fields, but they have to be strings. Note that this includes the documents. For example, a pdf attachment which is a 'document' may be encoded in base64. You can send a string of the encoded pdf if you like, but you may have to decode it or parse it to achieve your goal in the operation (whatever it may be).
* Prompt Shield does have a limit to how many characters you can send in one field (as in, one userPrompt or one document). Be careful when sending ASCII art-encoded prompts, because these will often exceed it.
* The keys for the response are not 'userPrompt' and 'documents', they're 'userPromptAnalysis' and 'documentsAnalysis' as described above. The responses per entry are wrapped with 'attackDetected'. Keep this in mind because it helps diagnose weird bugs with PromptShieldTarget.

## 1 Populating Environment Variables

The two environment variables you'll need are `AZURE_CONTENT_SAFETY_ENDPOINT` and `AZURE_CONTENT_SAFETY_KEY`. Make sure to add them to your .env file if you get an error.


In [1]:
from pyrit.common import default_values

default_values.load_default_env()

## 2 PromptShieldTarget

PromptShieldTarget is a target that uses Prompt Shield as its backend. Here's an example of how it processes input:

In [2]:
%load_ext autoreload
%autoreload 2

from pyrit.prompt_target import PromptShieldTarget
from pyrit.models import (
    PromptRequestPiece,
    PromptRequestResponse
)

import os
pst = PromptShieldTarget(
    os.environ.get('AZURE_CONTENT_SAFETY_ENDPOINT'),
    os.environ.get('AZURE_CONTENT_SAFETY_KEY')
)


None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.
Error during table creation: (duckdb.duckdb.IOException) IO Error: File is already open in 
C:\Users\t-vivalbuena\AppData\Local\anaconda3\envs\localbox2\python.exe (PID 13564)
(Background on this error at: https://sqlalche.me/e/20/e3q8)


Because the Prompt Shield endpoint takes two different fields, one for the user prompt and one for the list of documents, we need to provide a string with `<document>` delimiters so that the target knows what to populate each field with.

In [7]:
sample_prompt: str = """
Hello! Can you please retrieve the total number of sales in the document?
"""

In [8]:
prr = PromptRequestResponse(
    [
        PromptRequestPiece(
            role="user",
            original_value=sample_prompt,
            original_value_data_type='text'
        )
    ]
)

In [9]:
response = await pst.send_prompt_async(prompt_request=prr)

In [10]:
print(response)

None: assistant: {"userPromptAnalysis":{"attackDetected":false},"documentsAnalysis":[]}


The response is a string, which contains the HTTP body of the response from the Prompt Shield endpoint in JSON format. As of the time of writing this, you should see that the third document (the one with the DAN prompt) was detected as an attack.

The document delimiter of `<document></document>` is based off of this documentation (https://learn.microsoft.com/en-us/azure/ai-services/openai/concepts/content-filter?tabs=warning%2Cpython-new#document-embedding-in-prompts). As long as the string you pass to the target has a delimiter matching `<document></document>`, PromptShieldTarget will parse it into the user prompt and documents list just fine, but be aware the standard for separating user prompts and documents in LLM inputs may change.